In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [3]:
# Define constants
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
EPOCHS = 100
INITIAL_LEARNING_RATE = 0.001

In [ ]:
# Define the data generators with validation split
data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = data_gen.flow_from_directory(
    '/content/drive/MyDrive/Final_Year_Weather_Classification/Dataset/SkyFall',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Use 'categorical' for multi-label classification
    subset='training')

validation_generator = data_gen.flow_from_directory(
    '/content/drive/MyDrive/Final_Year_Weather_Classification/Dataset/SkyFall',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation')

# Split validation data into validation and test sets
test_generator = data_gen.flow_from_directory(
    '/content/drive/MyDrive/Final_Year_Weather_Classification/Dataset/SkyFall',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False)


In [ ]:
# Function to create model
def create_model(num_classes):
    input_tensor = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor)
    x = base_model.output
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    output_tensor = Dense(num_classes, activation='sigmoid')(x)  # Sigmoid activation for multi-label classification

    model = Model(inputs=input_tensor, outputs=output_tensor)
    return model

In [ ]:
# Function to compile and train model
def compile_and_train_model(model, train_generator, validation_generator, num_classes):
    model.compile(optimizer=Adam(learning_rate=INITIAL_LEARNING_RATE),
                  loss='binary_crossentropy',  # Binary cross-entropy for multi-label classification
                  metrics=['accuracy'])
    history = model.fit(train_generator,
                        epochs=EPOCHS,
                        validation_data=validation_generator)
    return history

In [ ]:
# Function to evaluate model
def evaluate_model(model, test_generator):
    test_generator.reset()
    predictions = model.predict(test_generator)
    y_pred = (predictions > 0.5).astype("int32")  # Thresholding predictions at 0.5
    y_true = test_generator.classes
    print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))
    print(confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1)))

In [ ]:


# Function to plot training history
def plot_history(history, model_name):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{model_name} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()

# Create and train models
# Adjust the number of classes based on your dataset
num_classes = len(train_generator.class_indices)
nightnet = create_model(num_classes=num_classes)
history_nightnet = compile_and_train_model(nightnet, train_generator, validation_generator, num_classes=num_classes)


In [ ]:
# Save the entire model as a H5 file
nightnet.save("/content/drive/MyDrive/Final_Year_Weather_Classification/CNN_Resuidal_Code/CNN_Skyfall.h5")
print("Model and weights saved.")

# To load the model and weights later, use:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model("/content/drive/MyDrive/Final_Year_Weather_Classification/CNN_Resuidal_Code/CNN_Skyfall.h5")
print("Model loaded successfully.")
